In [1]:
import numpy as np
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import scipy

In [5]:
df= pd.read_parquet("dataset2.parquet", engine="pyarrow")

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [10]:
def create_labels(data, n_labels):
    labels = [[] for i in range(len(data))]
    labels_names = list(data.columns[:n_labels])
    for i in range(len(data)):
        vector = list(data.iloc[i][:n_labels])
        for j, elt in enumerate(vector):
            if elt:
                labels[i].append(labels_names[j])
    return labels

In [11]:
def keep_unique_label_data(df):
    df_unique = df
    to_drop = []
    labels = create_labels(df_unique, 21)
    for i in range(len(labels)):
        if len(labels[i]) > 1:
            to_drop.append(i)
    df_unique.drop(to_drop, 0, inplace=True)
    return df_unique

In [3]:
import pandas as pd

df0 = pd.read_parquet("dataset2.parquet", engine="pyarrow")
df = df0.copy()

#Ajout colonne nombre de labels pour chaque titre
df["Nombre labels"] = df.iloc[:,0:21].sum(axis=1)

# df_unilabel: la base avec les titres ne comportant qu'un seul label
df_unilabel = df[df["Nombre labels"] == 1]
df_unilabel

,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,rock,...,jazz,electronic,african,country,song_index,audio_features,usage_features,artist_name,song_title,Nombre labels
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,195,"[0.7089933156967163, -0.6799708008766174, -0.3...","[-0.008137362965188247, -0.004262254068607103,...",Miles Davis,Générique,1
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,417,"[-0.19785162806510925, 0.023803016170859337, 0...","[0.00041993208544090015, 0.0001525178585704480...",Spin Doctors,Two Princes,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,530,"[-0.14142198860645294, 0.26422423124313354, 0....","[-0.009766703341977859, 0.0019064089803134893,...",Justin Bieber,One Less Lonely Girl,1
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,909,"[-0.07582522183656693, -0.42076417803764343, -...","[0.005023148381526522, 0.010775540737929749, 0...",Paul McCartney,My Love,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1257,"[-0.16135981678962708, -0.07056394964456558, -...","[0.00517276708433931, 0.006723451311827004, 0....",Jackson 5,Dancing Machine,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1312,"[-0.15023231506347656, 0.37152692675590515, 0....","[-0.0008908412643011058, 0.011325100149836086,...",Nightwish,Over The Hills And Far Away,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1878,"[-0.2080802023410797, 0.07194497436285019, 0.0...","[-0.003933269674219889, 0.00030428776516409033...",Eminem,My Name Is,1
9,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,2090,"[0.8002901673316956, -0.629996657371521, -0.30...","[-0.01022850556259187, -0.006592896674946462, ...",John Coltrane Quartet,Too Young To Go Steady,1
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2594,"[0.47648051381111145, 0.1992919147014618, 0.92...","[-0.002906493268587049, 0.0007957801676216864,...",Godsmack,I Stand Alone,1
12,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,3132,"[-0.18785671889781952, 0.31075695157051086, 0....","[0.0011409202772163327, 0.010546446125021267, ...",George Michael,Waiting for That Day,1


In [4]:
def transform_parquet_to_csv(file_path,name_csv):
    df=pd.read_parquet(file_path, engine="pyarrow")
    ind=df.index
    columns1=[]
    for i in range (1,257):
      col1= 'audio_feature_%i' % i
      columns1.append(col1)
    columns2=[]
    for i in range (1,129):
      col2= 'usage_feature_%i' % i
      columns2.append(col2)
    split_df_audio = pd.DataFrame(df['audio_features'].tolist(),columns=columns1,index=ind)
    split_df_usage=pd.DataFrame(df['usage_features'].tolist(),columns=columns2,index=ind)
    df1 = pd.concat([df, split_df_audio], axis=1)
    df =pd.concat([df1, split_df_usage], axis=1)
    df.drop(columns=['audio_features','usage_features'], inplace=True)
    df.set_index('song_index', inplace=True,drop=True)
    df.to_csv(name_csv)
    return df



In [12]:
transform_parquet_to_csv(keep_unique_label_data(df),'dataset.csv')

,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,rock,...,usage_feature_119,usage_feature_120,usage_feature_121,usage_feature_122,usage_feature_123,usage_feature_124,usage_feature_125,usage_feature_126,usage_feature_127,usage_feature_128
song_index,,,,,,,,,,,,,,,,,,,,,
195.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.008582,-0.000024,-0.272207,0.210503,0.030972,0.080964,-0.015423,0.102148,0.365081,-0.066897
417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.182309,-0.051225,-0.004617,-0.005726,-0.038334,-0.063972,0.053881,-0.001152,0.045032,0.027505
530.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.044620,-0.132282,-0.141684,-0.106271,-0.189065,-0.229804,0.083824,-0.160790,0.040417,-0.094503
909.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.127573,-0.068515,-0.106374,-0.102694,-0.066937,0.026911,-0.032158,0.063670,-0.026653,-0.068251
1257.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.024463,-0.052952,-0.059056,0.041670,-0.025673,-0.000446,-0.003438,0.060359,0.058938,0.029523
1312.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.033834,0.059702,0.127839,0.042218,-0.058068,-0.132771,0.048156,0.002076,0.139599,0.032241
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.017524,-0.030789,-0.035952,-0.053677,-0.015703,0.001386,0.011345,0.022860,0.039047,-0.013739
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.054643,0.010348,-0.306829,0.199233,0.056750,0.033910,-0.033535,0.071093,0.342383,-0.056093
1878.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.041783,-0.068033,0.020318,0.011393,0.057474,-0.006398,0.040105,-0.014617,-0.119008,-0.025669


In [13]:
df=pd.read_csv('dataset.csv')

In [14]:
df=df.fillna(0)

In [15]:
def extract_values_array(df,p,usage=False):
    n=df.shape[0]
    df_extracted= df.sample(int(p*n), random_state=0)
    df_extracted.drop(columns=['song_title','artist_name'],inplace=True)
    #pour qu'on ait des sorties similaires à travers plusieurs appels
    columns3=[]
    if (usage==False):
        for i in range (1,129):
            col2= 'usage_feature_%i' % i
            columns3.append(col2)
        df_extracted.drop(columns=columns3,inplace=True)
    l=list(df_extracted.columns[0:22])
    x=list(df_extracted.columns[22:])
    l.remove('song_index')
    X=df_extracted[x].values
    Y=df_extracted[l].values
    return(X,Y)

In [16]:
X,Y= extract_values_array(df,0.1)

In [17]:
df.head(5)

,song_index,asian,rnb,reggae,blues,pop,dance,folk,arabic-music,indie,...,usage_feature_119,usage_feature_120,usage_feature_121,usage_feature_122,usage_feature_123,usage_feature_124,usage_feature_125,usage_feature_126,usage_feature_127,usage_feature_128
0,195.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.008582,-0.000024,-0.272207,0.210503,0.030972,0.080964,-0.015423,0.102148,0.365081,-0.066897
1,417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.182309,-0.051225,-0.004617,-0.005726,-0.038334,-0.063972,0.053881,-0.001152,0.045032,0.027505
2,530.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.044620,-0.132282,-0.141684,-0.106271,-0.189065,-0.229804,0.083824,-0.160790,0.040417,-0.094503
3,909.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.127573,-0.068515,-0.106374,-0.102694,-0.066937,0.026911,-0.032158,0.063670,-0.026653,-0.068251
4,1257.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.024463,-0.052952,-0.059056,0.041670,-0.025673,-0.000446,-0.003438,0.060359,0.058938,0.029523


In [18]:
df.isnull().any()

song_index           False
asian                False
rnb                  False
reggae               False
blues                False
pop                  False
dance                False
folk                 False
arabic-music         False
indie                False
rock                 False
soulfunk             False
latin                False
classical            False
k-pop                False
brazilian            False
metal                False
rap                  False
jazz                 False
electronic           False
african              False
country              False
artist_name          False
song_title           False
audio_feature_1      False
audio_feature_2      False
audio_feature_3      False
audio_feature_4      False
audio_feature_5      False
audio_feature_6      False
                     ...  
usage_feature_99     False
usage_feature_100    False
usage_feature_101    False
usage_feature_102    False
usage_feature_103    False
usage_feature_104    False
u

In [19]:
import pandas as pd # Pour le dataframe
import numpy as np # Pour la normalisation et calculs de moyenne
import matplotlib.pyplot as plt # Pour la visualisation

from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve, RandomizedSearchCV 
# Split de dataset et optimisation des hyperparamètres
from sklearn.ensemble import RandomForestClassifier # Random forest

from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, zero_one_loss, classification_report 
# Métriques pour la mesure de performances

In [20]:
#  on affiche tous les paramètres qui peuvent être optimisée

rf = RandomForestClassifier(random_state = 0)
from pprint import pprint
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}


In [21]:
# Un random search permet de se faire une première idée des valeurs 
# optimales des différents hyper-paramètres, en balayant de façon très large 
# les différentes possibilités et en selectionnant les meilleures combinaison 
# par validation croisée.

# On affiche la grille à tester.

# nombre d'arbres
n_estimators = [500, 1000, 2000, 3000, 4000, 5000]
# profondeur max de l'arbre
max_depth = [20]
max_depth.append(None)
# nombre d'échantillon min nécessaire par noeuds
min_samples_split = [2, 4]#[2]
# nombre d'échantillon min nécessaire par feuilles
min_samples_leaf = [1, 2]#[1]

In [22]:
# création de la grille
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              }
pprint(random_grid)

{'max_depth': [20, None],
 'min_samples_leaf': [1, 2],
 'min_samples_split': [2, 4],
 'n_estimators': [500, 1000, 2000, 3000, 4000, 5000]}


In [23]:
# création du modèle
rf = RandomForestClassifier(random_state = 0, max_features = 'sqrt', bootstrap = True)

In [24]:
# random search
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=0, n_jobs = -1)

In [25]:
from sklearn.preprocessing import normalize, StandardScaler


In [26]:
# séparer les données en training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(X, Y, test_size = 0.25, random_state = 0)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

sc = StandardScaler()
train_features = sc.fit_transform(train_features)
test_features = sc.transform(test_features)

Training Features Shape: (3695, 256)
Training Labels Shape: (3695, 21)
Testing Features Shape: (1232, 256)
Testing Labels Shape: (1232, 21)


In [ ]:
# création du modèle
rf = RandomForestClassifier(random_state = 0, max_features = 'sqrt', bootstrap = True)

# random search
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=0, n_jobs = -1)

# fit le modèle
rf_random.fit(train_features, train_labels)

pd_res = pd.concat([pd.DataFrame(rf_random.cv_results_["params"]),pd.DataFrame(rf_random.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)
pd_res = pd_res.sort_values('Accuracy', ascending=False)
print(rf_random.best_params_)
pd_res.head(5)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [27]:
# création du modèle
rf = RandomForestClassifier(n_estimators=4000, max_features='sqrt', max_depth=20, min_samples_split=2, min_samples_leaf=1, bootstrap=True, criterion='gini' ,random_state=0)

# fit le modèle
rf.fit(train_features, train_labels)

# prédictions
predictions = rf.predict(test_features)

# Zero_one_loss error
errors = zero_one_loss(test_labels, predictions, normalize=False)
print('zero_one_loss error :', errors)

# Accuracy Score
accuracy_test = accuracy_score(test_labels, predictions)
print('accuracy_score on test dataset :', accuracy_test)

print(classification_report(predictions, test_labels))

zero_one_loss error : 1066
accuracy_score on test dataset : 0.13474025974025974
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
 

/Users/sirinelouati/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/Users/sirinelouati/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/sirinelouati/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Users/sirinelouati/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_f

In [28]:
import seaborn as sns

In [29]:
sns.set()
mat = confusion_matrix(test_labels, predictions)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, xticklabels=genres, yticklabels=genres)
plt.xlabel('true label')
plt.ylabel('predicted label')

ValueError: multilabel-indicator is not supported